<a href="https://colab.research.google.com/github/LianaN/TextLab/blob/main/FewShortLerning_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install openai
! pip install langchain
! pip install yt_dlp
! pip install pydub
! pip install gtts
!pip install git+https://github.com/huggingface/transformers sentencepiece datasets
!pip install --upgrade azure-ai-formrecognizer > /dev/null
!pip install --upgrade azure-cognitiveservices-speech > /dev/null
!pip install --upgrade azure-ai-vision > /dev/null

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-xu331zgg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-xu331zgg
  Resolved https://github.com/huggingface/transformers to commit 1c7e5e236823cd38faac8115f96205a82c17fff9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import os

import openai

from langchain.llms import OpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

from langchain.agents.agent_toolkits import AzureCognitiveServicesToolkit
from langchain.tools.azure_cognitive_services.text2speech import AzureCogsText2SpeechTool

from gtts import gTTS
from pydub import AudioSegment

In [5]:
os.environ["OPENAI_API_KEY"] = "..."
os.environ["AZURE_COGS_KEY"] = "..."
os.environ["AZURE_COGS_ENDPOINT"] = "..."
os.environ["AZURE_COGS_REGION"] = "..."

In [6]:
prank_examples = [
  {
    "request":
"""
Escribe un guión para una broma telefónica. Añade [clears throat] en algunas frases.

Aquí están los ajustes de la broma:
Tema: Llamada desde la policía sobre SEAT Ibiza azul
Nombre del bromista: Sargento García
Nombre de la persona a la que se le hace la broma: Juan
""",
    "prank":
"""
Bromista: ¡Hola!!!... uh... [clears throat] puedo hablar con el señor Juan?
Víctima: Sí, soy Juan. ¿Quién llama?
Bromista: Buenas tardes, señor Juan. [clears throat] Soy el Sargento García de la estación de policía local.
Víctima: Oh... ¿ocurre algo?
Bromista: Señor, ha habido un pequeño incidente que involucra un gato callejero y su vehículo. ¿Usted tiene un SEAT Ibiza azul?
Víctima: Sí, es correcto. ¿Qué pasó?
Bromista: A ver.. uh... este no es un gato común [clears throat]... Se trata de una raza especial, extremadamente rara y protegida, un Persa Azul... Parece que ha adoptado su coche como su territorio y ahora se niega a abandonarlo.
Víctima: ¿Qué? ¿Un gato... en mi coche?
Bromista: Está encima de su coche... Y, desafortunadamente, debido a su condición de raza protegida, legalmente no podemos retirarlo sin su permiso.
Víctima: Entonces, ¿qué quiere que haga al respecto?
Bromista: Bueno, señor, necesitamos que venga a la estación de policía local para firmar un formulario de liberación para poder reubicar al gato de manera segura.
Víctima: Me parece una locura...
Bromista: Es una situación bastante inusual. Gracias por comprender y cooperar.
"""
  },
  {
    "request":
"""
Escribe un guión para una broma telefónica. Añade [clears throat] en algunas frases.

Aquí están los ajustes de la broma:
Tema: Alquiler de barcos en el puerto de Mahón
Nombre del bromista: Sergio de la empresa Merak
Nombre de la persona a la que se le hace la broma: Ingrid
""",
    "prank":
"""
Bromista: ¡Hola!!! Soy Sergio... uh... de la empresa de alquiler de barcos Merak! Estamos en el puerto de Mahón.. uh..! ¿Estoy hablando con Ingrid???
Víctima: Sí-si, soy yo!
Bromista: ¡Hola, Ingrid! [clears throat] Quería confirmar su reserva para nuestro yate de lujo "El Dorado", para el próximo fin de semana. Tenemos todo listo, solo necesitamos su confirmación final.
Víctima: ¿Qué???? Yo no hice ninguna reserva...
Bromista: ¿En serio??? Tenemos aquí una reserva a su nombre, incluyendo un servicio completo de catering, una banda en vivo y un espectáculo de fuegos artificiales.
Víctima: NO, NO, es imposible.
Bromista: Como que no? Usted ha hecho la reserva hace 7 días. La reserva incluye todo, el alquiler del yate, la comida, la música, el seguro del barco!!
Víctima: NO, QUE raro! Yo no hice la reserva...
Bromista: ¿En serio? Vaya, esto es un problema. Nuestra política de cancelación es de 72 horas. Pasado ese tiempo, los gastos ya están comprometidos y no podemos hacer devoluciones...
Víctima: Pero es un error.
Bromista: Ummm... parece que hay un gran malentendido aquí, Ingrid. ¿Podría ser que alguien más hizo la reserva en tu nombre?
Víctima: No lo se...
Bromista: Sabes, esto suena muy extraño, pero... [clears throat] ¿Podría ser que tengas un admirador secreto que ha organizado una gran sorpresa para ti?
Víctima: Podría ser... Pero me temo que es un error.
Bromista: [clears throat] Bueno... deberías prepararte para una gran fiesta en el yate, entonces. Te esperamos mañana a las 9:00 en nuestra oficina del puerto de Mahon.

"""
  },
]

In [7]:
prank_prompt = PromptTemplate(input_variables=["request", "prank"], template="Request: {request}\n{prank}")
print(prank_prompt.format(**prank_examples[0]))

Request: 
Escribe un guión para una broma telefónica. Añade [clears throat] en algunas frases.

Aquí están los ajustes de la broma:
Tema: Llamada desde la policía sobre SEAT Ibiza azul
Nombre del bromista: Sargento García
Nombre de la persona a la que se le hace la broma: Juan


Bromista: ¡Hola!!!... uh... [clears throat] puedo hablar con el señor Juan?
Víctima: Sí, soy Juan. ¿Quién llama?
Bromista: Buenas tardes, señor Juan. [clears throat] Soy el Sargento García de la estación de policía local.
Víctima: Oh... ¿ocurre algo?
Bromista: Señor, ha habido un pequeño incidente que involucra un gato callejero y su vehículo. ¿Usted tiene un SEAT Ibiza azul?
Víctima: Sí, es correcto. ¿Qué pasó?
Bromista: A ver.. uh... este no es un gato común [clears throat]... Se trata de una raza especial, extremadamente rara y protegida, un Persa Azul... Parece que ha adoptado su coche como su territorio y ahora se niega a abandonarlo.
Víctima: ¿Qué? ¿Un gato... en mi coche?
Bromista: Está encima de su coch

In [8]:
prompt = FewShotPromptTemplate(
    examples=prank_examples,
    example_prompt=prank_prompt,
    suffix="Request: {request}",
    input_variables=["request"]
)

In [9]:
llm = OpenAI(temperature=0, max_tokens=2000)

In [14]:
chain = LLMChain(llm=llm, prompt=prompt)
dialogue = chain.run(
"""
Escribe un guión para una broma telefónica. Añade [clears throat] en algunas frases.

Aquí están los ajustes de la broma:
Tema: Llamada desde el restaurante Paput Menorca sobre la reserva para esta tarde
Nombre del bromista: Carlos
Nombre de la persona a la que se le hace la broma: Judith
""")
print(dialogue)


Bromista: ¡Hola! ¿Puedo hablar con Judith?
Víctima: Sí, soy yo. ¿Quién llama?
Bromista: Hola, Judith. [clears throat] Soy Carlos, del restaurante Paput Menorca. Estamos llamando para confirmar su reserva para esta tarde.
Víctima: ¿Mi reserva? No tengo ninguna reserva.
Bromista: [clears throat] ¿En serio? Tenemos aquí una reserva a su nombre para esta tarde. Estamos preparando una cena especial para usted, con un menú de degustación de 5 platos.
Víctima: ¿Qué? No tengo ninguna reserva.
Bromista: [clears throat] Bueno, esto es un poco extraño. ¿Podría ser que alguien más haya hecho la reserva en su nombre?
Víctima: No lo sé...
Bromista: [clears throat] Bueno, entonces, ¿quiere que cancelemos la reserva?
Víctima: No, espera... ¿Quién hizo la reserva?
Bromista: [clears throat] Bueno, según nuestros registros, la reserva fue hecha por un tal "Sr. Misterioso".
Víctima: ¿Sr. Misterioso?
Bromista: Sí, eso es lo que dice aquí. [clears throat] Así que, ¿quiere que cancelemos la reserva o quiere

In [12]:
dialogue = chain.run(
"""
Escribe un guión para una broma telefónica. Añade [clears throat] en algunas frases.

Aquí están los ajustes de la broma:
Tema: Llamada de una desconocida sobre la recogida de mejillones en la cala Padera
Nombre del bromista: Sofia
Nombre de la persona a la que se le hace la broma: Lidia
""")
print(dialogue)


Bromista: ¡Hola! ¿Estoy hablando con Lidia?
Víctima: Sí, soy yo. ¿Quién llama?
Bromista: Hola, Lidia. [clears throat] Soy Sofia, una desconocida que llama desde la cala Padera.
Víctima: ¿Qué quieres?
Bromista: Bueno, he estado recogiendo mejillones en la cala y me he encontrado con una gran cantidad de mejillones con tu nombre grabado en ellos.
Víctima: ¿Qué? ¿Cómo es posible?
Bromista: Bueno, parece que alguien ha estado dejando mejillones con tu nombre grabado en ellos en la cala durante los últimos meses. [clears throat] ¿Sabes algo al respecto?
Víctima: No, no tengo ni idea.
Bromista: Bueno, entonces, ¿quieres que te los envíe?
Víctima: ¿Enviarme los mejillones?
Bromista: Sí, ¿quieres que te los envíe? [clears throat] Parece que alguien quiere que los tengas.
Víctima: No, no es necesario.
Bromista: Bueno, entonces, ¿quieres que los regale a alguien?
Víctima: No, no es necesario.
Bromista: Bueno, entonces, ¿quieres que los comamos?
Víctima: No, no es necesario.
Bromista: [clears th

In [13]:
dialogue = chain.run(
"""
Escribe un guión para una broma telefónica. Añade [clears throat] en algunas frases.

Aquí están los ajustes de la broma:
Tema: Llamada de escuela online ILERNA. Ingrid no podrá realizar las prácticas de radiografía este año y tendrá que esperar hasta el año que viene.
Nombre del bromista: Carlos de ILERNA
Nombre de la persona a la que se le hace la broma: Ingrid
""")
print(dialogue)


Bromista: ¡Hola! [clears throat] ¿Estoy hablando con Ingrid?
Víctima: Sí, soy yo. ¿Quién llama?
Bromista: Hola Ingrid, soy Carlos de ILERNA. Estoy llamando para informarle que, lamentablemente, no podrá realizar las prácticas de radiografía este año.
Víctima: ¿Qué? ¿Por qué?
Bromista: Bueno, [clears throat] hemos recibido una notificación de la Junta de Radiología de España que indica que no hay suficientes radiólogos para cubrir todas las prácticas.
Víctima: ¿Y qué pasa conmigo?
Bromista: Bueno, [clears throat] tendrá que esperar hasta el año que viene para poder realizar las prácticas.
Víctima: Pero eso significa que tendré que retrasar mi graduación.
Bromista: Sí, eso es correcto. [clears throat] Lo sentimos mucho, pero no hay nada que podamos hacer al respecto.
Víctima: Esto es muy frustrante...
Bromista: Lo entendemos. [clears throat] Si hay algo que podamos hacer para ayudarla, no dude en contactarnos.


In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Hola Ingrid! Soy Carlos de ILEARNA", return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

In [ ]:
from IPython.display import Audio

Audio(speech.numpy(), rate=16000)